In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import time
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found")
from selenium import webdriver
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.
CHROMEDRIVER_PATH = 'E:/GOMMT/Solve for Tomorrow (6292020)/Selenium/chromedriver.exe'
driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)

No module named 'google' found


In [2]:
driver.get("https://www.easemytrip.com/")
driver.maximize_window()

In [3]:
button = driver.find_element_by_xpath('//*[@id="myTopnav"]/div[1]/ul/li[1]/a')
webdriver.ActionChains(driver).move_to_element(button).perform()
time.sleep(2)
webdriver.ActionChains(driver).click(button).perform()

In [4]:
# From https://www.easemytrip.com/flights.html
List = driver.find_element_by_xpath('/html/body/div[13]/div/div/ul')
allElements = List.find_elements_by_tag_name("a")
links = []
for element in allElements:
    links.append(element.get_attribute("href"))
links

['https://www.easemytrip.com/offers/coupon.html',
 'https://www.easemytrip.com/offers/new-user.html',
 'https://www.easemytrip.com/offers/coupon.html',
 'https://www.easemytrip.com/offers/new-user.html',
 'https://www.easemytrip.com/offers/coupon.html',
 'https://www.easemytrip.com/offers/new-user.html']

In [8]:
class coupon:
    def __init__(self, code, what_You_Get,How_do_you_get_it,What_else_do_you_need_to_know,Terms_and_Conditions):
        self.code = code
        self.what_You_Get = what_You_Get
        self.How_do_you_get_it = How_do_you_get_it
        self.What_else_do_you_need_to_know = What_else_do_you_need_to_know
        self.Terms_and_Conditions = Terms_and_Conditions

In [9]:
couponList = []
for link in links:
    driver.get(link)
    content = driver.page_source
    soup = BeautifulSoup(content)
    code = soup.find('div', attrs={'class':'cp_code'}).text
    i = 1
    ele = soup.findAll('ul', attrs={'class':'dot-list'})
    a = ''.join(ele[0].findAll(text=True)).strip()
    b = ''.join(ele[1].findAll(text=True)).strip()
    c = ''.join(ele[2].findAll(text=True)).strip()
    d = ''.join(ele[3].findAll(text=True)).strip()
    x = coupon(code,a,b,c,d)
    couponList.append(x)

In [15]:
for coupon in couponList:
    print(coupon.code)

EMT2020
EMTFIRST
EMT2020
EMTFIRST
EMT2020
EMTFIRST


In [ ]:
driver.get('https://flight.easemytrip.com/FlightList/Index?srch=DEL-Delhi-India|BOM-Mumbai-India|09/07/2020&px=1-0-0&cbn=0&ar=undefined&isow=true&isdm=true&lng=&')
